# Amazon Bedrock AgentCore Runtime で Strands Agent を Instana Observability と共に使用

## 概要

このノートブックでは、Instana 可観測性統合を使用して Strands エージェントを Amazon Bedrock AgentCore Runtime にデプロイする方法を説明します。この実装は Amazon Bedrock モデルを使用し、OpenTelemetry (OTEL) を通じてテレメトリデータを Instana に送信します。

## 主なコンポーネント

- **Strands Agents**: 組み込みテレメトリサポートを持つ LLM 駆動エージェントを構築するための Python フレームワーク
- **Amazon Bedrock AgentCore Runtime**: AWS でエージェントをホスティングおよびスケーリングするためのマネージドランタイムサービス
- **Instana**: OTEL 経由でトレースを受信するアプリケーション向けリアルタイム可観測性およびパフォーマンス監視プラットフォーム
- **OpenTelemetry**: テレメトリデータの収集とエクスポートのための業界標準プロトコル

## アーキテクチャ

エージェントはコンテナ化され、AgentCore Runtime にデプロイされ、呼び出し用の HTTP エンドポイントを提供します。テレメトリデータは Strands エージェントから OTEL エクスポーターを通じて Instana に流れ、監視とデバッグに使用されます。この実装では、代わりに Instana を使用するために AgentCore のデフォルトの可観測性を無効にしています。

## 前提条件

- Python 3.10+
- [Amazon Bedrock AgentCore の開始方法](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agentcore-get-started-toolkit.html#agentcore-get-started-prerequisites)
- Bedrock および [AgentCore 権限](https://docs.aws.amazon.com/aws-managed-policy/latest/reference/BedrockAgentCoreFullAccess.html)を持つ AWS 認証情報
- [Instana](https://www.ibm.com/products/instana) アカウント
- Docker がローカルにインストールされていること
- Amazon Bedrock モデルへのアクセス

### 正しい Instana エンドポイントの見つけ方

Instana インスタンスの正しい OTLP エンドポイントを見つけるには：

1. Instana で**サイドバー**を開きます。
2. 一番下までスクロールし、**About Instana** を選択します。
3. 表示される **Instance Region** をメモします。
4. このリージョンに基づいて、エンドポイント[ドキュメント](https://www.ibm.com/docs/en/instana-observability/1.0.309?topic=instana-backend)を使用して適切な **HTTP OTLP エンドポイント (4318)** を選択します。


### Instana キーの取得

Instana キーを取得するには：

1. サイドバーから **Agents & Collectors** をクリックします。
2. **Linux – Automatic Installation (One-liner)** を選択します。
3. `-a` フラグの後に表示されるキーをコピーします — これが **Agent Key** であり、**Instana Key** として使用されます。

以下の画像を参照してください。

<div style="text-align:left">
    <img src="../images/instana_agent_key.png" width="99%"/>
</div>

### プロジェクトのルートディレクトリに `.env` という名前の新しいファイルを作成し、以下の環境変数を追加します：

以下のセルを実行して、プロジェクトのルートに .env ファイルを自動的に生成します。ファイルが作成されたら、プレースホルダーの値を実際の Instana OTLP エンドポイントと API キーで更新してください。

`.env` ファイルは、設定の詳細（API キーやエンドポイントなど）を安全に保存するために使用され、ハードコーディングせずにアプリケーションに簡単にロードできます。

**注意**: `.env` ファイルを GitHub にコミットしたり、Instana キーを公開で共有したりしないでください。資格情報を安全に保つため、`.gitignore` ファイルに `.env` を追加してください。

### .env ファイルの内容例：

In [ ]:
%%writefile .env

# Instana OTLP エンドポイント（例：https://otlp-blue-saas.instana.io:4318）
OTEL_EXPORTER_OTLP_ENDPOINT="<instana_endpoint>"

# 注意：この例では StrandsTelemetry() を使用してエクスポートしており、HTTP OTLP エンドポイントのみをサポートしています。
# Strands Telemetry を使用する場合は、必ず HTTP エンドポイントを使用してください。

# Instana キー
INSTANA_KEY="<agent_key>"

以下のセルを実行して依存関係をインストールします：

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## AWS 認証情報の設定

ノートブックを実行する前に、`前提条件`セクションで提供されているリンクに従って AWS 認証情報を設定してください。

## エージェント実装

エージェントファイル（`strands_nova.py`）は、Web 検索機能を持つトラベルエージェントを実装しています。主な設定は以下を含みます：
- Strands テレメトリの初期化

In [ ]:
%%writefile strands_nova.py
import os
import logging
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent, tool
from strands.models import BedrockModel
from strands.telemetry import StrandsTelemetry
from ddgs import DDGS

logging.basicConfig(level=logging.ERROR, format="[%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(os.getenv("AGENT_RUNTIME_LOG_LEVEL", "INFO").upper())


@tool
def web_search(query: str) -> str:
    """
    DuckDuckGo を使用して Web から情報を検索します。

    Args:
        query: 検索クエリ

    Returns:
        検索結果を含む文字列
    """
    try:
        ddgs = DDGS()
        results = ddgs.text(query, max_results=5)

        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(
                f"{i}. {result.get('title', 'No title')}\n"
                f"   {result.get('body', 'No summary')}\n"
                f"   Source: {result.get('href', 'No URL')}\n"
            )

        return "\n".join(formatted_results) if formatted_results else "No results found."

    except Exception as e:
        return f"Error searching the web: {str(e)}"

# Bedrock モデルを初期化する関数
def get_bedrock_model():
    region = os.getenv("AWS_DEFAULT_REGION", "us-east-1")
    model_id = os.getenv("BEDROCK_MODEL_ID", "amazon.nova-lite-v1:0")

    bedrock_model = BedrockModel(
        model_id=model_id,
        region_name=region,
        temperature=0.0,
        max_tokens=1024
    )
    return bedrock_model

# Bedrock モデルを初期化
bedrock_model = get_bedrock_model()

# エージェントのシステムプロンプトを定義
system_prompt = """あなたはリアルタイムの Web 情報にアクセスできる、パーソナライズされた旅行推奨を専門とする経験豊富なトラベルエージェントです。
あなたの役割は、Web 検索を使用して最新情報を取得し、ユーザーの好みに合った理想の目的地を見つけることです。
最新情報、簡潔な説明、実用的な旅行の詳細を含む包括的な推奨を提供してください。"""

app = BedrockAgentCoreApp()

def initialize_agent():
    """適切なテレメトリ設定でエージェントを初期化します。"""

    # サードパーティ設定で Strands テレメトリを初期化
    strands_telemetry = StrandsTelemetry()
    strands_telemetry.setup_otlp_exporter()
    
    # エージェントを作成してキャッシュ
    agent = Agent(
        model=bedrock_model,
        system_prompt=system_prompt,
        tools=[web_search]
    )
    
    return agent

@app.entrypoint
def strands_agent_bedrock(payload, context=None):
    """
    ペイロードでエージェントを呼び出します
    """
    user_input = payload.get("prompt")
    logger.info("[%s] User input: %s", context.session_id, user_input)
    
    # 適切な設定でエージェントを初期化
    agent = initialize_agent()
    
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

### AgentCore Runtime デプロイの設定

次に、スターターツールキットを使用して AgentCore Runtime デプロイを設定します。
このステップでは、エントリポイント、先ほど作成した実行ロール、および requirements ファイルを設定します。
また、起動時に Amazon ECR リポジトリを自動的に作成するようにスターターキットを設定します。

設定コマンドを実行すると、アプリケーションコードに基づいて Dockerfile が自動的に生成されます。

**注意：**
bedrock_agentcore_starter_toolkit はデフォルトで AgentCore Observability を有効にします。
Instana を可観測性に使用する場合は、次のセクションで説明するように、デフォルトの AgentCore Observability 設定を削除する必要があります。

<div style="text-align:left">
    <img src="../images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_instana_observability"
response = agentcore_runtime.configure(
    entrypoint="strands_nova.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    disable_otel=True,
)
response

### 代替設定（事前作成済み IAM ロールの使用）

AWS アカウントで**自動ロール作成が許可されていない**場合や、**権限をより細かく制御したい**場合は、ツールキットに自動作成させる代わりに**事前作成済みの IAM ロール**を使用できます。

この場合：
- `auto_create_execution_role=False` を設定
- `execution_role` パラメータで既存の IAM ロール ARN を指定

このアプローチは以下の場合に便利です：
- 組織が**最小権限の IAM ポリシー**を強制している
- 複数の AgentCore デプロイで**共通の実行ロールを再利用**したい
- 新しいロールの作成が許可されていない**制限された環境**（企業や共有 AWS アカウントなど）で作業している

手動で管理される IAM ロールでランタイムを設定する方法については、以下のサンプルを参照してください。

In [ ]:
# 代替設定（オプション）

from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

ROLE_ARN = "your_IAM_role"

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_instana_observability"
response = agentcore_runtime.configure(
    entrypoint="strands_nova.py",
    auto_create_execution_role=False,   # 自動ロール作成を無効化
    execution_role=ROLE_ARN,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_instana_observability",
    disable_otel=True,
)
response

## AgentCore Runtime へのデプロイ

Dockerfile が生成されたので、エージェントを AgentCore Runtime に起動しましょう。

このステップでは自動的に以下が行われます：
- Amazon ECR リポジトリの作成（存在しない場合）
- コンテナ化されたエージェントを AgentCore Runtime 環境にデプロイ
- 先ほど作成した `.env` ファイルから環境変数（Instana エンドポイントと API キーなど）をロード

**注意：** このセルを実行する前に、`.env` ファイルが正しく設定されていることを確認してください。そうでないと、テレメトリデータは Instana にエクスポートされません。

<div style="text-align:left">
    <img src="../images/launch.png" width="75%"/>
</div>

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Fetch configuration
otel_endpoint = os.getenv("OTEL_EXPORTER_OTLP_ENDPOINT")
instana_key = os.getenv("INSTANA_KEY")

# Format Instana header
otel_auth_header = f"x-instana-key={instana_key}"

# Launch the AgentCore runtime
launch_result = agentcore_runtime.launch(
    env_vars={
        "BEDROCK_MODEL_ID": "amazon.nova-lite-v1:0",
        "OTEL_EXPORTER_OTLP_ENDPOINT": otel_endpoint,
        "OTEL_EXPORTER_OTLP_HEADERS": otel_auth_header,
        "OTEL_SERVICE_NAME": "AWS-APP",
        "OTEL_EXPORTER_OTLP_INSECURE": "false",
        "DISABLE_ADOT_OBSERVABILITY": "true",
    }
)

launch_result

## デプロイステータスの確認

デプロイが開始されると、AgentCore Runtime のセットアップが完了するまで数分かかる場合があります。
以下のコードを使用して、デプロイステータスをリアルタイムで監視できます。

このスニペットは、数秒ごとに AgentCore エンドポイントのステータスを継続的にチェックし、最終状態に達するまで続けます：
- READY → デプロイが正常に完了
- CREATE_FAILED、UPDATE_FAILED、または DELETE_FAILED → デプロイでエラーが発生

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtime の呼び出し

最後に、サンプルプロンプトでデプロイしたエージェントを呼び出して、レスポンスをテストし、期待通りに動作しているか確認できます。

<div style="text-align:left">
    <img src="../images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "I'm planning a weekend trip to london. What are the must-visit places and local food I should try?"})

以下のコードを使用して、エージェントの出力を見やすく表示します。

In [ ]:
from IPython.display import Markdown, display
display(Markdown("".join(invoke_response['response'])))

## Instana でトレースを表示

トレースを表示するには：
1. Instana ダッシュボードにアクセス
2. サイドバーから `Analytics` をクリック
3. `Hidden calls` タブの下にある `Show internal calls` チェックボックスをオン
4. `Add Filter` をクリックし、`Service Name` を選択
5. 検索バーで "strands-agents" を検索
6. クリックして関連する呼び出しのリストを表示および分析。各呼び出しはユーザーインタラクションを表します
7. 任意の呼び出しをクリックして完全なトレースデータを表示

トレースには以下が含まれます：
- エージェント呼び出しの詳細
- ツール呼び出し（Web 検索）
- レイテンシとトークン使用量を含むモデルインタラクション
- リクエスト/レスポンスペイロード

## クリーンアップ（オプション）

テストが完了したら、以下のコードを使用して AgentCore Runtime と関連する Amazon ECR リポジトリを削除し、不要なリソース使用とコストを回避してください。

In [ ]:
import boto3

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

## まとめ

Instana 可観測性を使用して Strands エージェントを Amazon Bedrock AgentCore Runtime に正常にデプロイしました。この実装は以下を示しています：
- Strands エージェントと AgentCore Runtime の統合
- トレースを Instana に送信するための OpenTelemetry の設定
- テレメトリ設定を確実にするための適切な初期化順序

エージェントは、Instana を通じた完全な可観測性を備えたマネージドでスケーラブルな環境で実行されています。